In [28]:
library(lme4)
library(lmerTest)
library(nlme)
library(ggplot2)
library(grid)
library(gridExtra)
library(lattice)
library(plyr)
library(MASS)
library(stats4)
library(dplyr)
library(car)
library(tidyr)
library(broom)


# Results from the sDDM modelling for decision consistency analysis
Based on the data collected in June 2022

## Load data and preprocessing

In [2]:
# trials table
data_raw <- read.table("/Users/hutianqi/Desktop/Project Cognitive Imprecision/Z1E Dataset_Base NT/Trials_22S1.csv", header=TRUE, sep=",")

data_raw$logRT <- log(1 + data_raw$RT)

# make a copy of the data where less than 300ms and more than 10s rounds are rejected
data1 <- subset(data_raw, RT > .3 & RT < 10)

# # create another dataset in wh subjects with error_choice > 5 are rejected
# data2 <- subset(data1, error_choice <= 5)


In [3]:
names(data1)

[1] "subject"        "trial"          "treatment"      "lot_code"      
 [5] "display"        "risk_level"     "risk"           "reward"        
 [9] "certainty"      "WTP"            "choice"         "RT"            
[13] "slc_lot"        "correct"        "incorrect"      "value_diff"    
[17] "value_sum"      "lottery_favour" "lot_correct"    "error_choice"  
[21] "decmode"        "reflection"     "intuition"      "BNT"           
[25] "NFC"            "FI"             "age"            "sex_code"      
[29] "edu_code"       "d"              "inv_d"          "B"             
[33] "nondectime"     "logRT"

In [4]:
# individual difference table
idt1 <- read.table("/Users/hutianqi/Desktop/Project Cognitive Imprecision/Z1E Dataset_Base NT/IDT_22S1.csv", header=TRUE, sep=",")

# idt2 <- subset(idt1, error_choice <= 5)


In [5]:
names(idt1)

[1] "subject"      "treatment"    "age"          "sex"          "sex_code"    
 [6] "education"    "edu_code"     "reflection"   "intuition"    "BNT"         
[11] "NFC"          "FI"           "decmode"      "accuracy"     "lottery_freq"
[16] "mean_RT"      "error_choice" "d"            "inv_d"        "B"           
[21] "nondectime"   "ddm_sample"

###  All continuous variables are scaled due to the inclusion of interactive terms and to avoid the error "Model failed to converge with max|grad|".

# Predicting decision consistency


In [6]:
# All subjects
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20616.0  20655.5 -10303.0  20606.0    19909 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0051 -0.6020 -0.4189  0.7759 16.0584 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.539    0.7341  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.23399    0.07719 -15.987  < 2e-16 ***
scale(value_diff)            -0.84458    0.02701 -31.271  < 2e-16 ***
scale(WTP)                    0.19150    0.02005   9.551  < 2e-16 ***
scale(value_diff):scale(WTP)  0.15215    0.01871   8.131 4.24e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effect

In [7]:
# All subjects; additional demographic factors as control variables 
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + sex_code + scale(age) + scale(edu_code) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + sex_code + scale(age) +  
    scale(edu_code) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20618.1  20681.2 -10301.0  20602.1    19906 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0143 -0.6017 -0.4188  0.7768 16.0560 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.5163   0.7186  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.09551    0.10719 -10.220  < 2e-16 ***
scale(value_diff)            -0.84501    0.02701 -31.283  < 2e-16 ***
scale(WTP)                    0.19160    0.02005   9.557  < 2e-16 ***
sex_code                     -0.27466    0.15058  -1.824   0.0681 .  
scale(age)                   -0.01934    0.0751

### Because demographic factors are not significant predictors, they are dopped in folowing analysis.

In [8]:
# All subjects; with Psychometrics
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(reflection) +  
    scale(BNT) + scale(NFC) + scale(FI) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20611.8  20682.9 -10296.9  20593.8    19905 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0211 -0.6014 -0.4192  0.7749 16.0140 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.4703   0.6858  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.238495   0.072480 -17.087  < 2e-16 ***
scale(value_diff)            -0.845119   0.027017 -31.281  < 2e-16 ***
scale(WTP)                    0.190060   0.020051   9.479  < 2e-16 ***
scale(reflection)            -0.188765   0.091639  -2.060   0.0394 *  
scale(BNT)                

In [9]:
# All subjects; with Psychometrics one by one
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(reflection) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(BNT) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(NFC) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(FI) + (1|subject), data1, family=binomial(link = "logit")))



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(reflection) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20611.5  20658.9 -10299.8  20599.5    19908 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0188 -0.6010 -0.4186  0.7767 16.0132 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.5013   0.7081  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.23572    0.07463 -16.557  < 2e-16 ***
scale(value_diff)            -0.84499    0.02702 -31.276  < 2e-16 ***
scale(WTP)                    0.19045    0.02005   9.498  < 2e-16 ***
scale(reflection)            -0.19102    0.07377  -2.589  0.00962 ** 
scale(value_diff):scale(WTP)  0.15260    0.01872   8.152 3.57e-16 ***

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(BNT) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20617.0  20664.4 -10302.5  20605.0    19908 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0044 -0.6013 -0.4189  0.7754 16.0701 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.5327   0.7299  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.23407    0.07676 -16.076  < 2e-16 ***
scale(value_diff)            -0.84480    0.02701 -31.273  < 2e-16 ***
scale(WTP)                    0.19121    0.02005   9.535  < 2e-16 ***
scale(BNT)                   -0.07447    0.07511  -0.991    0.321    
scale(value_diff):scale(WTP)  0.15238    0.01872   8.142 3.88e-16 ***
---
Si

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(NFC) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20611.4  20658.8 -10299.7  20599.4    19908 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0065 -0.6013 -0.4190  0.7749 16.0651 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.5009   0.7078  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.23711    0.07461 -16.580  < 2e-16 ***
scale(value_diff)            -0.84482    0.02701 -31.280  < 2e-16 ***
scale(WTP)                    0.19105    0.02004   9.531  < 2e-16 ***
scale(NFC)                   -0.19387    0.07429  -2.610  0.00907 ** 
scale(value_diff):scale(WTP)  0.15199    0.01871   8.125 4.48e-16 ***
---
Si

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(FI) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20618.0  20665.4 -10303.0  20606.0    19908 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0051 -0.6018 -0.4189  0.7764 16.0614 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.5389   0.7341  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.23408    0.07718 -15.989  < 2e-16 ***
scale(value_diff)            -0.84455    0.02701 -31.269  < 2e-16 ***
scale(WTP)                    0.19142    0.02005   9.545  < 2e-16 ***
scale(FI)                     0.01416    0.07673   0.185    0.854    
scale(value_diff):scale(WTP)  0.15217    0.01871   8.132 4.22e-16 ***
---
Sig

In [10]:
# All subjects; with DDM parameters
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(inv_d) + scale(B) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(inv_d)*scale(B) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
incorrect ~ scale(value_diff) * scale(WTP) + scale(inv_d) + scale(B) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20490.2  20545.5 -10238.1  20476.2    19907 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6545 -0.6030 -0.4138  0.7800 15.3248 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.1126   0.3356  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.24676    0.03957 -31.507  < 2e-16 ***
scale(value_diff)            -0.83681    0.02683 -31.185  < 2e-16 ***
scale(WTP)                    0.17588    0.01993   8.827  < 2e-16 ***
scale(inv_d)                  0.63637    0.03909  16.278  < 2e-16 ***
scale(B)                     -0.12357    0.03768  -3.279  0.00

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
incorrect ~ scale(value_diff) * scale(WTP) + scale(inv_d) * scale(B) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20490.3  20553.5 -10237.2  20474.3    19906 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6549 -0.6035 -0.4141  0.7800 15.3022 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.1099   0.3316  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                             Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.24844    0.03925 -31.809  < 2e-16 ***
scale(value_diff)            -0.83637    0.02683 -31.172  < 2e-16 ***
scale(WTP)                    0.17497    0.01993   8.777  < 2e-16 ***
scale(inv_d)                  0.63952    0.03881  16.476  < 2e-16 ***
scale(B)                     -0.13633    0.03845  -3.545 0.000

In [11]:
# All subjects; with Psychometrics; with DDM parameters
summary(glmer(incorrect ~ scale(value_diff)*scale(WTP) + scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + scale(inv_d) + scale(B) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: incorrect ~ scale(value_diff) * scale(WTP) + scale(reflection) +  
    scale(BNT) + scale(NFC) + scale(FI) + scale(inv_d) + scale(B) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 20495.2  20582.1 -10236.6  20473.2    19903 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6505 -0.6025 -0.4140  0.7801 15.3406 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 0.1079   0.3286  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  -1.247067   0.038964 -32.005  < 2e-16 ***
scale(value_diff)            -0.837474   0.026848 -31.193  < 2e-16 ***
scale(WTP)                    0.174346   0.019962   8.734  < 2e-16 ***
scale(reflection)            -0.013949   0.049376  -0.283  0.7775

# RT analysis

In [12]:
# all subjects 
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + (1|subject), data1, family = Gamma(link = "identity")))
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + (1|subject), data1, family = Gamma(link = "identity")))

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58986.5  59034.0 -29487.3  58974.5    20128 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6261 -0.2555  0.3064 12.5628 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2531   0.5031  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526727   0.087768  28.789  < 2e-16 ***
scale(value_diff)            -0.086138   0.006776 -12.712  < 2e-16 ***
scale(WTP)                   -0.004463   0.008330  -0.536    0.592    
scale(value_diff):scale(WTP)  0.034525   0.005717   6.039 1.55e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58988.5  59043.9 -29487.3  58974.5    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6260 -0.2555  0.3064 12.5649 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2531   0.5031  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5265488  0.0881272  28.669  < 2e-16 ***
scale(value_diff)            -0.0861035  0.0069138 -12.454  < 2e-16 ***
scale(WTP)                   -0.0044901  0.0083931  -0.535    0.593    
slc_lot                       0.0003785  0.0153292   0.025    0.980    
scale(value_diff):scale(WTP)  0.0344976 

In [13]:
# all subjects; DDM parameters
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(inv_d) + scale(B) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(inv_d)*scale(B) + (1|subject), data1, family = Gamma(link = "identity")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(inv_d) + scale(B) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58826.0  58889.3 -29405.0  58810.0    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.6172 -0.6327 -0.2535  0.3071 12.6094 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.04419  0.2102  
 Residual             0.30953  0.5564  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.462962   0.034133  72.158  < 2e-16 ***
scale(value_diff)            -0.086557   0.006768 -12.789  < 2e-16 ***
scale(WTP)                   -0.004082   0.008291  -0.492    0.622    
scale(inv_d)                  0.182221   0.033959   5.366 8.05e-08 ***
scale(B)               

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(inv_d) * scale(B) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58823.9  58895.1 -29402.9  58805.9    20125 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.6202 -0.6321 -0.2545  0.3062 12.7027 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.04213  0.2053  
 Residual             0.30946  0.5563  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.460974   0.033267  73.976  < 2e-16 ***
scale(value_diff)            -0.086586   0.006767 -12.796  < 2e-16 ***
scale(WTP)                   -0.004279   0.008290  -0.516   0.6057    
scale(inv_d)                  0.180190   0.033123   5.440 5.33e-08 ***
scale(B)               

In [14]:
# all subjects; error choices
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + error_choice + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + error_choice + (1|subject), data1, family = Gamma(link = "identity")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + error_choice + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58985.8  59041.1 -29485.9  58971.8    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5973 -0.6260 -0.2555  0.3050 12.5832 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2465   0.4964  
 Residual             0.3147   0.5610  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.612668   0.100553  25.983  < 2e-16 ***
scale(value_diff)            -0.086049   0.006777 -12.697  < 2e-16 ***
scale(WTP)                   -0.004381   0.008331  -0.526   0.5990    
error_choice                 -0.033131   0.019853  -1.669   0.0952 .  
scale(value_diff):scale(WTP)  0.034532  

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + error_choice +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58987.8  59051.0 -29485.9  58971.8    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5973 -0.6261 -0.2554  0.3051 12.5865 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2465   0.4964  
 Residual             0.3147   0.5610  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.6123897  0.1008522  25.903  < 2e-16 ***
scale(value_diff)            -0.0859976  0.0069150 -12.436  < 2e-16 ***
scale(WTP)                   -0.0044218  0.0083932  -0.527   0.5983    
slc_lot                       0.0005812  0.0153295   0.038   0.9698    
error_choice       

In [15]:
# all subjects; with Psychometrics
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + (1|subject), data1, family = Gamma(link = "identity")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: 
RT ~ scale(value_diff) * scale(WTP) + scale(reflection) + scale(BNT) +  
    scale(NFC) + scale(FI) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58990.2  59069.3 -29485.1  58970.2    20124 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5972 -0.6261 -0.2553  0.3049 12.5788 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2425   0.4925  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526947   0.085813  29.447  < 2e-16 ***
scale(value_diff)            -0.086182   0.006776 -12.718  < 2e-16 ***
scale(WTP)                   -0.004331   0.008331  -0.520   0.6032    
scale(reflection)            -0.064989   0.109559  -0.593   0

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + scale(reflection) +  
    scale(BNT) + scale(NFC) + scale(FI) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58992.1  59079.2 -29485.1  58970.1    20123 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5973 -0.6260 -0.2552  0.3049 12.5816 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2425   0.4925  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5266925  0.0861684  29.323  < 2e-16 ***
scale(value_diff)            -0.0861380  0.0069140 -12.458  < 2e-16 ***
scale(WTP)                   -0.0043654  0.0083937  -0.520   0.6030    
slc_lot                       0.0005143  0.01532

In [16]:
# all subjects; with Psychometrics one by one; with slc_lot
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + scale(reflection) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + scale(BNT) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + scale(NFC) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + scale(FI) + (1|subject), data1, family = Gamma(link = "identity")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + scale(reflection) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58990.5  59053.7 -29487.2  58974.5    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5975 -0.6261 -0.2554  0.3062 12.5691 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2530   0.5030  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5266943  0.0881046  28.678  < 2e-16 ***
scale(value_diff)            -0.0860978  0.0069140 -12.453  < 2e-16 ***
scale(WTP)                   -0.0044790  0.0083932  -0.534    0.594    
slc_lot                       0.0003903  0.0153292   0.025    0.980    
scale(reflecti

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + scale(BNT) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58990.2  59053.5 -29487.1  58974.2    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5975 -0.6260 -0.2553  0.3063 12.5698 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2522   0.5022  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5264888  0.0879660  28.721  < 2e-16 ***
scale(value_diff)            -0.0861029  0.0069140 -12.453  < 2e-16 ***
scale(WTP)                   -0.0044609  0.0083935  -0.531    0.595    
slc_lot                       0.0003903  0.0153291   0.025    0.980    
scale(BNT)           

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + scale(NFC) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58990.5  59053.8 -29487.3  58974.5    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6260 -0.2555  0.3065 12.5640 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2531   0.5031  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5264508  0.0881340  28.666  < 2e-16 ***
scale(value_diff)            -0.0861065  0.0069139 -12.454  < 2e-16 ***
scale(WTP)                   -0.0044889  0.0083931  -0.535    0.593    
slc_lot                       0.0003719  0.0153292   0.024    0.981    
scale(NFC)           

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + scale(FI) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58986.8  59050.1 -29485.4  58970.8    20126 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6261 -0.2556  0.3044 12.5798 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2442   0.4941  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                               Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.5266142  0.0864635  29.222  < 2e-16 ***
scale(value_diff)            -0.0861302  0.0069139 -12.458  < 2e-16 ***
scale(WTP)                   -0.0043884  0.0083935  -0.523   0.6011    
slc_lot                       0.0005188  0.0153291   0.034   0.9730    
scale(FI)             

In [17]:
# all subjects; with Psychometrics one by one; without slc_lot
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(reflection) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(BNT) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(NFC) + (1|subject), data1, family = Gamma(link = "identity")))

summary(glmer(RT ~ scale(value_diff)*scale(WTP) + scale(FI) + (1|subject), data1, family = Gamma(link = "identity")))



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(reflection) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58988.5  59043.8 -29487.2  58974.5    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5975 -0.6260 -0.2555  0.3063 12.5670 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2530   0.5030  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526900   0.087767  28.791  < 2e-16 ***
scale(value_diff)            -0.086133   0.006776 -12.711  < 2e-16 ***
scale(WTP)                   -0.004455   0.008331  -0.535    0.593    
scale(reflection)             0.020697   0.087360   0.237    0.813    
scale(value_diff):scale(WTP) 

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(BNT) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58988.2  59043.6 -29487.1  58974.2    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5975 -0.6261 -0.2554  0.3062 12.5676 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2522   0.5022  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526686   0.087612  28.839  < 2e-16 ***
scale(value_diff)            -0.086139   0.006776 -12.712  < 2e-16 ***
scale(WTP)                   -0.004434   0.008331  -0.532    0.595    
scale(BNT)                    0.048803   0.086214   0.566    0.571    
scale(value_diff):scale(WTP)  0.034520   0

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(NFC) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58988.5  59043.9 -29487.3  58974.5    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6260 -0.2555  0.3066 12.5620 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2531   0.5031  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526658   0.087776  28.785  < 2e-16 ***
scale(value_diff)            -0.086141   0.006776 -12.712  < 2e-16 ***
scale(WTP)                   -0.004464   0.008330  -0.536    0.592    
scale(NFC)                   -0.004547   0.088231  -0.052    0.959    
scale(value_diff):scale(WTP)  0.034525   0

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + scale(FI) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58984.8  59040.2 -29485.4  58970.8    20127 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5974 -0.6260 -0.2555  0.3042 12.5769 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2441   0.4941  
 Residual             0.3145   0.5608  
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.526878   0.086108  29.345  < 2e-16 ***
scale(value_diff)            -0.086176   0.006776 -12.717  < 2e-16 ***
scale(WTP)                   -0.004353   0.008331  -0.523   0.6013    
scale(FI)                    -0.167003   0.085904  -1.944   0.0519 .  
scale(value_diff):scale(WTP)  0.034503   0.

In [18]:
# all subjects; adding the variable "correct" –– whether the higher values option is chosen.
summary(glmer(RT ~ scale(value_diff)*scale(WTP) + slc_lot + correct + (1|subject), data1, family = Gamma(link = "identity")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: Gamma  ( identity )
Formula: RT ~ scale(value_diff) * scale(WTP) + slc_lot + correct + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 58270.4  58333.6 -29127.2  58254.4    19906 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.5978 -0.6241 -0.2540  0.3036 11.7179 

Random effects:
 Groups   Name        Variance Std.Dev.
 subject  (Intercept) 0.2511   0.5011  
 Residual             0.3133   0.5597  
Number of obs: 19914, groups:  subject, 97

Fixed effects:
                              Estimate Std. Error t value Pr(>|z|)    
(Intercept)                   2.628581   0.088760  29.614  < 2e-16 ***
scale(value_diff)            -0.072174   0.007156 -10.085  < 2e-16 ***
scale(WTP)                   -0.010450   0.008419  -1.241    0.214    
slc_lot                       0.018995   0.015595   1.218    0.223    
correct                      

# Predict error choices

In [19]:
# by DDM parameters
summary(lm(error_choice ~ scale(inv_d) + scale(B), idt1))

summary(lm(error_choice ~ scale(inv_d)*scale(B), idt1))



Call:
lm(formula = error_choice ~ scale(inv_d) + scale(B), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-5.399 -2.440 -1.112  1.061 15.324 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)    2.6082     0.4147   6.289 9.97e-09 ***
scale(inv_d)   1.4925     0.4171   3.578  0.00055 ***
scale(B)      -0.9519     0.4171  -2.282  0.02475 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.084 on 94 degrees of freedom
Multiple R-squared:  0.1565,	Adjusted R-squared:  0.1386 
F-statistic: 8.723 on 2 and 94 DF,  p-value: 0.000335



Call:
lm(formula = error_choice ~ scale(inv_d) * scale(B), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-5.215 -2.435 -1.182  1.173 15.198 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)             2.6015     0.4167   6.243 1.27e-08 ***
scale(inv_d)            1.5062     0.4199   3.587 0.000536 ***
scale(B)               -0.9845     0.4246  -2.318 0.022623 *  
scale(inv_d):scale(B)   0.1910     0.4088   0.467 0.641403    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.101 on 93 degrees of freedom
Multiple R-squared:  0.1585,	Adjusted R-squared:  0.1314 
F-statistic:  5.84 on 3 and 93 DF,  p-value: 0.001064


In [20]:
# by psychometrics
summary(lm(error_choice ~ scale(reflection) + scale(BNT) + scale(NFC) + scale(FI), idt1))

summary(lm(error_choice ~ scale(reflection), idt1))

summary(lm(error_choice ~ scale(BNT), idt1))

summary(lm(error_choice ~ scale(NFC), idt1))

summary(lm(error_choice ~ scale(FI), idt1))



Call:
lm(formula = error_choice ~ scale(reflection) + scale(BNT) + 
    scale(NFC) + scale(FI), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-5.772 -2.580 -1.454  1.853 14.496 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)         2.6082     0.4143   6.295 1.03e-08 ***
scale(reflection)  -0.6584     0.5334  -1.234  0.22020    
scale(BNT)         -0.2117     0.5172  -0.409  0.68327    
scale(NFC)         -1.0877     0.4310  -2.523  0.01334 *  
scale(FI)           1.2403     0.4364   2.842  0.00552 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.081 on 92 degrees of freedom
Multiple R-squared:  0.176,	Adjusted R-squared:  0.1402 
F-statistic: 4.912 on 4 and 92 DF,  p-value: 0.001235



Call:
lm(formula = error_choice ~ scale(reflection), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-4.910 -2.214 -1.674  1.090 16.326 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)         2.6082     0.4334   6.018 3.29e-08 ***
scale(reflection)  -1.1558     0.4356  -2.653  0.00935 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.268 on 95 degrees of freedom
Multiple R-squared:  0.06898,	Adjusted R-squared:  0.05918 
F-statistic: 7.038 on 1 and 95 DF,  p-value: 0.009352



Call:
lm(formula = error_choice ~ scale(BNT), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-3.870 -2.704 -2.122  1.296 15.296 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.6082     0.4442   5.872  6.3e-08 ***
scale(BNT)   -0.6553     0.4465  -1.468    0.145    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.374 on 95 degrees of freedom
Multiple R-squared:  0.02217,	Adjusted R-squared:  0.01188 
F-statistic: 2.154 on 1 and 95 DF,  p-value: 0.1455



Call:
lm(formula = error_choice ~ scale(NFC), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-4.277 -2.651 -1.616  1.166 15.349 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.6082     0.4388   5.943 4.59e-08 ***
scale(NFC)   -0.9378     0.4411  -2.126   0.0361 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.322 on 95 degrees of freedom
Multiple R-squared:  0.04542,	Adjusted R-squared:  0.03537 
F-statistic:  4.52 on 1 and 95 DF,  p-value: 0.0361



Call:
lm(formula = error_choice ~ scale(FI), data = idt1)

Residuals:
   Min     1Q Median     3Q    Max 
-4.931 -2.722 -1.722  1.507 14.088 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.6082     0.4321   6.036 3.05e-08 ***
scale(FI)     1.2000     0.4344   2.763  0.00689 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.256 on 95 degrees of freedom
Multiple R-squared:  0.07436,	Adjusted R-squared:  0.06462 
F-statistic: 7.632 on 1 and 95 DF,  p-value: 0.006886


# Predicting DDM parameters - cognitive uncertainty and response caution

## Predicting cognitive uncertainty

In [21]:
# by psychometrics
summary(lm(inv_d ~ scale(reflection) + scale(BNT) + scale(NFC) + scale(FI), idt1))

# by psychometrics and demographic factors
summary(lm(inv_d ~ scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + scale(age) + sex_code + scale(edu_code), idt1))



Call:
lm(formula = inv_d ~ scale(reflection) + scale(BNT) + scale(NFC) + 
    scale(FI), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.198420 -0.036705  0.001091  0.053041  0.127805 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       -0.126898   0.007082 -17.917   <2e-16 ***
scale(reflection) -0.018283   0.009118  -2.005   0.0479 *  
scale(BNT)         0.009787   0.008841   1.107   0.2712    
scale(NFC)        -0.017570   0.007368  -2.385   0.0191 *  
scale(FI)         -0.002891   0.007460  -0.387   0.6993    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.06975 on 92 degrees of freedom
Multiple R-squared:  0.1167,	Adjusted R-squared:  0.07825 
F-statistic: 3.037 on 4 and 92 DF,  p-value: 0.02118



Call:
lm(formula = inv_d ~ scale(reflection) + scale(BNT) + scale(NFC) + 
    scale(FI) + scale(age) + sex_code + scale(edu_code), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.211008 -0.043520  0.002326  0.054034  0.141098 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)       -0.1164779  0.0102825 -11.328   <2e-16 ***
scale(reflection) -0.0166442  0.0093148  -1.787   0.0774 .  
scale(BNT)         0.0096267  0.0089389   1.077   0.2844    
scale(NFC)        -0.0156218  0.0076223  -2.049   0.0434 *  
scale(FI)         -0.0024776  0.0074956  -0.331   0.7418    
scale(age)        -0.0045466  0.0072822  -0.624   0.5340    
sex_code          -0.0206270  0.0147194  -1.401   0.1646    
scale(edu_code)    0.0003394  0.0075022   0.045   0.9640    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.06995 on 89 degrees of freedom
Multiple R-squared:  0.1407,	Adjusted R-squared:  0

In [22]:
# by psychometrics one by one
summary(lm(inv_d ~ scale(reflection), idt1))

summary(lm(inv_d ~ scale(BNT), idt1))

summary(lm(inv_d ~ scale(NFC), idt1))

summary(lm(inv_d ~ scale(FI), idt1))



Call:
lm(formula = inv_d ~ scale(reflection), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.229054 -0.041514 -0.000795  0.054153  0.136368 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)       -0.126898   0.007279  -17.43   <2e-16 ***
scale(reflection) -0.013900   0.007317   -1.90   0.0605 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07169 on 95 degrees of freedom
Multiple R-squared:  0.0366,	Adjusted R-squared:  0.02646 
F-statistic: 3.609 on 1 and 95 DF,  p-value: 0.06049



Call:
lm(formula = inv_d ~ scale(BNT), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.208160 -0.042272 -0.001342  0.058890  0.126855 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.1268977  0.0074156 -17.112   <2e-16 ***
scale(BNT)  -0.0002896  0.0074542  -0.039    0.969    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07304 on 95 degrees of freedom
Multiple R-squared:  1.589e-05,	Adjusted R-squared:  -0.01051 
F-statistic: 0.001509 on 1 and 95 DF,  p-value: 0.9691



Call:
lm(formula = inv_d ~ scale(NFC), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.180138 -0.043152  0.007714  0.049465  0.119622 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.126898   0.007121 -17.820  < 2e-16 ***
scale(NFC)  -0.020274   0.007158  -2.832  0.00564 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07013 on 95 degrees of freedom
Multiple R-squared:  0.07787,	Adjusted R-squared:  0.06816 
F-statistic: 8.022 on 1 and 95 DF,  p-value: 0.005642



Call:
lm(formula = inv_d ~ scale(FI), data = idt1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.207590 -0.043169 -0.003479  0.055825  0.129593 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.126898   0.007410 -17.125   <2e-16 ***
scale(FI)   -0.002854   0.007448  -0.383    0.702    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07298 on 95 degrees of freedom
Multiple R-squared:  0.001543,	Adjusted R-squared:  -0.008967 
F-statistic: 0.1468 on 1 and 95 DF,  p-value: 0.7025


## Predicting response caution

In [23]:
# by psychometrics
summary(lm(B ~ scale(reflection) + scale(BNT) + scale(NFC) + scale(FI), idt1))

# by psychometrics and demographic factors
summary(lm(B ~ scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + scale(age) + sex_code + scale(edu_code), idt1))



Call:
lm(formula = B ~ scale(reflection) + scale(BNT) + scale(NFC) + 
    scale(FI), data = idt1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.74914 -0.21373  0.00489  0.16127  0.78091 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        1.361301   0.029513  46.126   <2e-16 ***
scale(reflection)  0.022152   0.037995   0.583   0.5613    
scale(BNT)         0.003194   0.036842   0.087   0.9311    
scale(NFC)         0.024122   0.030703   0.786   0.4341    
scale(FI)         -0.059457   0.031088  -1.913   0.0589 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2907 on 92 degrees of freedom
Multiple R-squared:  0.05685,	Adjusted R-squared:  0.01584 
F-statistic: 1.386 on 4 and 92 DF,  p-value: 0.2448



Call:
lm(formula = B ~ scale(reflection) + scale(BNT) + scale(NFC) + 
    scale(FI) + scale(age) + sex_code + scale(edu_code), data = idt1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.76199 -0.19353 -0.00456  0.16658  0.80907 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        1.381143   0.043066  32.070   <2e-16 ***
scale(reflection)  0.024797   0.039013   0.636   0.5267    
scale(BNT)         0.007293   0.037439   0.195   0.8460    
scale(NFC)         0.021847   0.031924   0.684   0.4955    
scale(FI)         -0.057881   0.031394  -1.844   0.0686 .  
scale(age)         0.032972   0.030500   1.081   0.2826    
sex_code          -0.039280   0.061649  -0.637   0.5257    
scale(edu_code)    0.004884   0.031421   0.155   0.8768    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.293 on 89 degrees of freedom
Multiple R-squared:  0.07313,	Adjusted R-squared:  0.0002254 
F-statistic: 1

In [24]:
# by psychometrics one by one
summary(lm(B ~ scale(reflection), idt1))

summary(lm(B ~ scale(BNT), idt1))

summary(lm(B ~ scale(NFC), idt1))

summary(lm(B ~ scale(FI), idt1))




Call:
lm(formula = B ~ scale(reflection), data = idt1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.76946 -0.18327 -0.00663  0.17086  0.89872 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)        1.36130    0.02964  45.926   <2e-16 ***
scale(reflection)  0.03889    0.02979   1.305    0.195    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2919 on 95 degrees of freedom
Multiple R-squared:  0.01762,	Adjusted R-squared:  0.007281 
F-statistic: 1.704 on 1 and 95 DF,  p-value: 0.1949



Call:
lm(formula = B ~ scale(BNT), data = idt1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.79033 -0.19292 -0.02022  0.17439  0.89526 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.36130    0.02984  45.622   <2e-16 ***
scale(BNT)   0.01958    0.02999   0.653    0.516    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2939 on 95 degrees of freedom
Multiple R-squared:  0.004465,	Adjusted R-squared:  -0.006015 
F-statistic: 0.4261 on 1 and 95 DF,  p-value: 0.5155



Call:
lm(formula = B ~ scale(NFC), data = idt1)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.77443 -0.18491 -0.03313  0.19107  0.86715 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.36130    0.02986  45.589   <2e-16 ***
scale(NFC)   0.01606    0.03002   0.535    0.594    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2941 on 95 degrees of freedom
Multiple R-squared:  0.003005,	Adjusted R-squared:  -0.00749 
F-statistic: 0.2863 on 1 and 95 DF,  p-value: 0.5938



Call:
lm(formula = B ~ scale(FI), data = idt1)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.8304 -0.2146 -0.0046  0.1836  0.7358 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.36130    0.02927   46.51   <2e-16 ***
scale(FI)   -0.06002    0.02942   -2.04   0.0441 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2883 on 95 degrees of freedom
Multiple R-squared:  0.04196,	Adjusted R-squared:  0.03188 
F-statistic: 4.161 on 1 and 95 DF,  p-value: 0.04414


# Prdicting the probability of choosing the lottery options

In [25]:
# All subjects
summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19178.0  19217.6  -9584.0  19168.0    20129 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-9.963 -0.568  0.181  0.578 99.715 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.319    1.148   
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.11074    0.11831   0.936    0.349    
scale(lottery_favour)             2.22430    0.03684  60.370   <2e-16 ***
scale(WTP)                       -0.22990    0.02285 -10.062   <2e-16 ***
scale(lottery_favour):scale(WTP) -0.40234    0.02352 -17.109   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of 

In [26]:
# All subjects; with Psychometrics
summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + scale(reflection) + scale(BNT) + scale(NFC) + scale(FI) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + scale(reflection) +  
    scale(BNT) + scale(NFC) + scale(FI) + (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19183.3  19254.5  -9582.7  19165.3    20125 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-9.997 -0.567  0.181  0.578 99.510 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.281    1.132   
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.10786    0.11668   0.924    0.355    
scale(lottery_favour)             2.22346    0.03685  60.338   <2e-16 ***
scale(WTP)                       -0.23048    0.02285 -10.086   <2e-16 ***
scale(reflection)                -0.11128    0.14858  -0.749    0.454    
scale(BNT)         

In [27]:
# All subjects; with Psychometrics one by one
summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + scale(reflection) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + scale(BNT) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + scale(NFC) + (1|subject), data1, family=binomial(link = "logit")))

summary(glmer(slc_lot ~ scale(lottery_favour)*scale(WTP) + scale(FI) + (1|subject), data1, family=binomial(link = "logit")))


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + scale(reflection) +  
    (1 | subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19179.6  19227.0  -9583.8  19167.6    20128 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-9.975 -0.568  0.181  0.577 99.574 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.312    1.145   
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.10999    0.11804   0.932    0.351    
scale(lottery_favour)             2.22407    0.03685  60.361   <2e-16 ***
scale(WTP)                       -0.23006    0.02285 -10.069   <2e-16 ***
scale(reflection)                -0.08142    0.11734  -0.694    0.488    
scale(lottery_favour):scale(WTP) -0.40232    0.02351 -17.

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + scale(BNT) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19179.9  19227.4  -9584.0  19167.9    20128 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-9.949 -0.568  0.181  0.578 99.744 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.317    1.148   
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.11080    0.11822   0.937    0.349    
scale(lottery_favour)             2.22439    0.03684  60.373   <2e-16 ***
scale(WTP)                       -0.22984    0.02285 -10.059   <2e-16 ***
scale(BNT)                        0.04056    0.11626   0.349    0.727    
scale(lottery_favour):scale(WTP) -0.40237    0.02352 -17.111   <

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + scale(NFC) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19179.7  19227.2  -9583.9  19167.7    20128 

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-9.970 -0.568  0.181  0.578 99.737 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.315    1.147   
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.10965    0.11812   0.928    0.353    
scale(lottery_favour)             2.22431    0.03684  60.372   <2e-16 ***
scale(WTP)                       -0.22988    0.02285 -10.061   <2e-16 ***
scale(NFC)                       -0.06519    0.11824  -0.551    0.581    
scale(lottery_favour):scale(WTP) -0.40251    0.02352 -17.116   <

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: slc_lot ~ scale(lottery_favour) * scale(WTP) + scale(FI) + (1 |  
    subject)
   Data: data1

     AIC      BIC   logLik deviance df.resid 
 19178.7  19226.1  -9583.3  19166.7    20128 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-10.011  -0.567   0.181   0.577  99.594 

Random effects:
 Groups  Name        Variance Std.Dev.
 subject (Intercept) 1.299    1.14    
Number of obs: 20134, groups:  subject, 97

Fixed effects:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                       0.11000    0.11743   0.937    0.349    
scale(lottery_favour)             2.22349    0.03685  60.341   <2e-16 ***
scale(WTP)                       -0.23044    0.02285 -10.083   <2e-16 ***
scale(FI)                         0.13884    0.11699   1.187    0.235    
scale(lottery_favour):scale(WTP) -0.40175    0.02352 -1